In [13]:
import sys
import re
import requests

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts.base import BaseOutputParser

sys.path.insert(0, r'C:\Users\HARSH\Documents\chatweb3-backend')
from utils import *
set_api_key()

In [8]:
TEMPLATE = \
'''
Query: {query}
Note: The query is related to the blockchain ecosystem.
Generate 10 related searches most asked on the web. 
1.'''

class ChatOutputParser(BaseOutputParser):

    @property
    def _type(self) -> str:
        """Return the type key."""
        return "chat_output_parser"

    def parse(self, text: str) -> str:
        """Parse the output of an LLM call."""
        text = text.strip()
        text = re.sub(r"\d+. ", '', text)
        text = text.split('\n')
        return text

In [9]:

class SearchURLs():
    def __init__(self,) -> None:
        super().__init__()
        llm = OpenAI(
            temperature=1.0, max_tokens=-1,
        )
        output_parser = ChatOutputParser()
        relSearch_prompt = PromptTemplate(
            input_variables=["query"],
            template=TEMPLATE,
            output_parser=output_parser,
        )
        self.chain = LLMChain(llm=llm, prompt=relSearch_prompt, verbose=False)

        subscription_key="df01e65a4da8434b841e8a81b34c9d25"
        self.search_url = "https://api.bing.microsoft.com/v7.0/search"
        self.headers = {"Ocp-Apim-Subscription-Key" : subscription_key}

    def via_llm(self, query): 
        relSearches = self.chain.apply_and_parse([{"query": query}])[0]
        return relSearches

    def via_bing(self, query):
        query = query+" Note: The query is related to the blockchain ecosystem."
        params  = {"q": query, "textDecorations": True, "textFormat": "HTML"}
        response = requests.get(self.search_url, headers=self.headers, params=params)
        response.raise_for_status()
        response = response.json()
        urls = [x['url'] for x in response['webPages']['value']]
        try: relSearches = [r['text'] for r in response['relatedSearches']['value']]
        except: relSearches = []
        return urls, relSearches

In [10]:
with open("basic_questions.txt") as file:
    queries = [line.rstrip() for line in file]

In [11]:
def find_urls(query, searcher, depth):
    urls, relS_bing = searcher.via_bing(query)
    urls = [u for u in urls if u not in all_urls]
    all_urls.update(set(urls))
    relS_llm = searcher.via_llm(query)
    print({query: urls})
    if depth==0: return {query: urls}
    relS = relS_bing + relS_llm
    q_urls = []
    for q in relS:
        if q in all_queries: continue
        x = find_urls(q, searcher, depth=depth-1)
        all_queries.add(q)
        q_urls.append(x)
    return q_urls

In [12]:
searcher = SearchURLs()
all_queries, all_urls = set(), set()
for q in queries:
    urls = find_urls(q, searcher, depth=5)
    break

{'What is web3?': ['https://hbr.org/2022/05/what-is-web3', 'https://ethereum.org/en/web3/', 'https://www.gemini.com/cryptopedia/what-is-web3-explained-blockchain-and-web3-crypto-projects', 'https://chain.link/education/web3', 'https://byjus.com/current-affairs/web-3-0/', 'https://consensys.net/blog/blockchain-explained/what-is-web3-here-are-some-ways-to-explain-it-to-a-friend/', 'https://crypto-current.co/what-are-web3-ecosystems/', 'https://www.blockchain-council.org/blockchain/blockchain-technology-in-web-3-0/', 'https://www.geeksforgeeks.org/what-is-blockchain-ecosystem/', 'https://101blockchains.com/blockchain-ecosystem/']}
{'What is web3.js?': ['https://web3js.org/', 'https://blog.logrocket.com/ethereum-blockchain-development-using-web3-js/', 'https://web3.hashnode.com/what-is-web3js-an-introduction-into-the-web3js-libraries', 'https://www.blockchain-council.org/web-3/what-is-web3-js-and-how-does-it-work/', 'https://moralis.io/web3-and-javascript-what-is-javascript-and-web3-js/', 

Retrying langchain.llms.openai.BaseOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-mrK2kSCEDDLFwshwzA1wjEBX on tokens per min. Limit: 250000 / min. Current: 324680 / min. Contact support@openai.com if you continue to have issues..


{'What is the best desktop Ethereum wallet?': ['https://codeless.co/best-ethereum-wallets/']}
{'What is the best mobile Ethereum wallet?': []}
{'What is MetaMask wallet for Ethereum?': ['https://decrypt.co/resources/metamask', 'https://101blockchains.com/metamask/', 'https://changelly.com/blog/what-is-metamask/', 'https://www.coinspeaker.com/guides/what-is-metamask-and-how-to-use-it/', 'https://dev.to/metamask/a-guide-to-metamask-ecosystem-leading-ethereum-blockchain-wallet-59k7', 'https://geekflareae.pages.dev/posts/what-is-metamask-and-how-to-use-it-/', 'https://cloudname.com/blog/what-is-metamask/', 'https://thefrisky.com/ethereum-and-metamask/', 'https://wikibinance.com/en/knowledge/metamask.html', 'https://informastre.com/what-is-metamask/']}
{'How to use MetaMask wallet for Ethereum?': ['https://www.sitepoint.com/metamask-ethereum-blockchain/', 'https://geekflare.com/beginners-guide-to-metamask/', 'https://frontierprotocols.com/how-to-use-metamask/', 'https://screenrant.com/creat

Retrying langchain.llms.openai.BaseOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-mrK2kSCEDDLFwshwzA1wjEBX on tokens per min. Limit: 250000 / min. Current: 324540 / min. Contact support@openai.com if you continue to have issues..
Retrying langchain.llms.openai.BaseOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-mrK2kSCEDDLFwshwzA1wjEBX on tokens per min. Limit: 250000 / min. Current: 405640 / min. Contact support@openai.com if you continue to have issues..


{'What type of wallet should I use for Ethereum? ': ['https://www.dummies.com/article/business-careers-money/personal-finance/cryptocurrency/different-types-of-ethereum-wallets-263674/']}
{'Best Ethereum Wallet': ['https://changelly.com/blog/best-ethereum-wallets-2/', 'https://blog.chainalysis.com/reports/cryptocurrency-ecosystem-bitcoin-ethereum-stablecoins/', 'https://coinloan.io/blog/ethereum-blockchain-ecosystem/']}
{'What is the safest Ethereum Wallet': ['https://trading-education.com/is-ethereum-safe']}
{'Ethereum Hardware Wallet': ['https://www.alibabacloud.com/blog/how-ethereum-bip-32-hardware-digital-wallet-works_597788', 'https://ethereum.org/en/', 'https://ethereum.org/en/developers/docs/data-and-analytics/', 'https://medium.com/blockchannel/tools-and-technologies-in-the-ethereum-ecosystem-e5b7e5060eb9']}
{'How to Store Ethereum': ['https://www.geeksforgeeks.org/how-to-store-data-in-blockchain/', 'https://www.geeksforgeeks.org/how-to-store-data-on-ethereum-blockchain/', 'htt

...

Retrying langchain.llms.openai.BaseOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-mrK2kSCEDDLFwshwzA1wjEBX on tokens per min. Limit: 250000 / min. Current: 324600 / min. Contact support@openai.com if you continue to have issues..


{'What can Ethereum be used for?': ['https://ethereum.org/en/developers/docs/intro-to-ether/', 'https://www.gemini.com/cryptopedia/ethereum-smart-contracts-tokens-use-cases', 'https://medium.com/cryptocurrency-secrets-for-beginners/ethereum-what-are-some-of-the-uses-for-ethereum-and-what-can-it-be-used-for-319ab3c0cc48', 'https://blog.devgenius.io/decoding-the-science-of-ethereum-its-top-use-cases-d2fba40c0cbb']}
{'How does Ethereum work?': ['https://www.makeuseof.com/what-is-ethereum-how-does-it-work/', 'https://blog.carret.in/what-is-ethereum-and-its-ecosystem/', 'https://ethereum.org/en/governance/', 'https://www.elluminatiinc.com/ethereum/', 'https://www.quora.com/How-does-one-access-the-Ethereum-blockchain-using-Python-language']}
{'What is the difference between Bitcoin and Ethereum?': ['https://www.thebalancemoney.com/ethereum-vs-bitcoin-how-do-eth-and-btc-compare-5218717', 'https://www.forbes.com/sites/bernardmarr/2018/02/05/what-is-the-difference-between-bitcoin-and-ethereum/'

Retrying langchain.llms.openai.BaseOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-mrK2kSCEDDLFwshwzA1wjEBX on tokens per min. Limit: 250000 / min. Current: 324660 / min. Contact support@openai.com if you continue to have issues..


{'What is the purpose of Ethereum smart contracts? ': ['https://www.geeksforgeeks.org/smart-contracts-in-blockchain/', 'https://www.blockchain-council.org/ethereum/smart-contracts-blockchain/', 'https://www.simplilearn.com/tutorials/blockchain-tutorial/what-is-smart-contract']}
{'How do Ethereum smart contracts work? ': ['https://www.coindesk.com/learn/how-do-ethereum-smart-contracts-work/', 'https://www.quicknode.com/guides/smart-contract-development/an-overview-of-how-smart-contracts-work-on-ethereum', 'https://www.techviral.org/ethereum-smart-contracts/', 'https://www.blockchain-council.org/ethereum/smart-contracts-works/', 'https://ark-invest.com/articles/analyst-research/smart-contracts-work/']}
{'What are the benefits of Ethereum smart contracts? ': ['https://content.enkronos.com/what-is-a-smart-contract-advantages-and-disadvantages/', 'https://merehead.com/blog/what-blockchain-smart-contracts-benefits/', 'https://www.techtarget.com/searchcio/feature/Examples-of-smart-contracts-o

HTTPError: 403 Client Error: Quota Exceeded for url: https://api.bing.microsoft.com/v7.0/search?q=What+is+an+ERCToken%3F++Note%3A+The+query+is+related+to+the+blockchain+ecosystem.&textDecorations=True&textFormat=HTML

In [ ]:
urls

[{'What is the purpose of web3?': ['https://hbr.org/2022/05/what-is-web3',
   'https://ethereum.org/en/web3/',
   'https://en.wikipedia.org/wiki/Web3',
   'https://www.nytimes.com/interactive/2022/03/18/technology/web3-definition-internet.html',
   'https://www.pcmag.com/how-to/what-is-web3-and-how-will-it-work',
   'https://www.thedrum.com/opinion/2022/02/09/what-web3-and-why-does-it-matter',
   'https://www.gemini.com/cryptopedia/what-is-web3-explained-blockchain-and-web3-crypto-projects',
   'https://www.drishtiias.com/daily-updates/daily-news-analysis/web-3-0',
   'https://byjus.com/current-affairs/web-3-0/',
   'https://consensys.net/blog/blockchain-explained/what-is-web3-here-are-some-ways-to-explain-it-to-a-friend/']},
 {'What is the role of web3 in blockchain?': ['https://101blockchains.com/blockchain-in-web-3-0/',
   'https://hbr.org/2022/05/what-is-web3',
   'https://blog.gaurav-dhiman.com/what-is-web3-and-role-of-blockchain-and-crypto-in-it',
   'https://www.blockchain-counc